Using Python 2.7. To get the dependencies installed

```sudo pip install beautifulsoup```

```sudo pip install pandas```

```sudo pip install nominatim```


In [1]:
from datetime import datetime
from  urllib import urlopen

from bs4 import BeautifulSoup
import pandas as pd


In [2]:
base_url = "http://www.nuforc.org/webreports/"
index_url = "http://www.nuforc.org/webreports/ndxevent.html"

In [3]:
source = BeautifulSoup(urlopen(index_url), "html5lib")

In [8]:
# source? - docstring
# source?? - entire code
#dir(source) - what methods the object has

In [16]:
monthly_urls =[]
for link in source.select("a")[1:13]:
    url = base_url + link["href"]
    monthly_urls.append(url)

In [17]:
monthly_urls

[u'http://www.nuforc.org/webreports/ndxe201703.html',
 u'http://www.nuforc.org/webreports/ndxe201702.html',
 u'http://www.nuforc.org/webreports/ndxe201701.html',
 u'http://www.nuforc.org/webreports/ndxe201612.html',
 u'http://www.nuforc.org/webreports/ndxe201611.html',
 u'http://www.nuforc.org/webreports/ndxe201610.html',
 u'http://www.nuforc.org/webreports/ndxe201609.html',
 u'http://www.nuforc.org/webreports/ndxe201608.html',
 u'http://www.nuforc.org/webreports/ndxe201607.html',
 u'http://www.nuforc.org/webreports/ndxe201606.html',
 u'http://www.nuforc.org/webreports/ndxe201605.html',
 u'http://www.nuforc.org/webreports/ndxe201604.html']

In [25]:
def scrape_month(url):
    source = BeautifulSoup(urlopen(url), "html5lib")
    rows = source.select("tr")
    data = []
    for row in rows:
        #print row
        cells = row.select("td")
        if len(cells) ==0:
            continue
        datestr = cells[0].text
        location = cells[1].text
        region = cells[2].text
        shape = cells[3].text
        duration = cells[4].text
        data_tuple = (datestr, location, region, shape, duration)
        data.append(data_tuple)
    return data
    

In [26]:
all_data = []
for url in monthly_urls:
    print "Processing {}".format(url)
    data = scrape_month(url)
    all_data = all_data + data

Processing http://www.nuforc.org/webreports/ndxe201703.html
Processing http://www.nuforc.org/webreports/ndxe201702.html
Processing http://www.nuforc.org/webreports/ndxe201701.html
Processing http://www.nuforc.org/webreports/ndxe201612.html
Processing http://www.nuforc.org/webreports/ndxe201611.html
Processing http://www.nuforc.org/webreports/ndxe201610.html
Processing http://www.nuforc.org/webreports/ndxe201609.html
Processing http://www.nuforc.org/webreports/ndxe201608.html
Processing http://www.nuforc.org/webreports/ndxe201607.html
Processing http://www.nuforc.org/webreports/ndxe201606.html
Processing http://www.nuforc.org/webreports/ndxe201605.html
Processing http://www.nuforc.org/webreports/ndxe201604.html


In [30]:
import pandas as pd

In [39]:
ufos_df = pd.DataFrame(all_data)
ufos_df.columns = ["datestr", "city", "state", "shape", "duration_description"]
ufos_df.head()
ufos_df.to_csv("raw_data.csv", index=False, encoding='utf-8' )

In [40]:
ufos_df.head()

,datestr,city,state,shape,duration_description
0,3/23/17 07:50,Asheville,NC,Cigar,2 minutes
1,3/22/17 22:25,Frederick,MD,Formation,4-5 seconds
2,3/22/17 22:15,Dover,NH,Triangle,5 minutes
3,3/22/17 21:30,North Richland Hills,TX,Light,30 seconds
4,3/22/17 20:00,Sheboygan,WI,Fireball,5-10 minutes


In [42]:
# IDEAS: how to augment the data
# get the coordinates using city and state - need an extra data source
# get the end time - need to parse the data properly
# we could try to get reference pictures for UFOs shaped like fireballs, triangles etc

In [49]:
import geocoder
# For reference: Ways to do the same thing in pandas, ie
# apply a function to a column

def to_lowercase(str):
    return str.lower()

ufos_df["shape"].apply(lambda x: x.lower())
ufos_df["shape"].apply(lambda x: to_lowercase(x))
ufos_df["shape"].apply(lambda shp: to_lowercase(shp))
ufos_df["shape"].apply(to_lowercase)

for i, row in ufos_df.head().iterrows():
    row["shape"] = to_lowercase(row["shape"])


In [50]:
#How to do an apply to an entire row
# ufos_df.apply(function, axis=1)

In [63]:
def create_location_str(row):
   # print row # TODO: actually combine
   # print row.__class__ # it's a pandas series
    return row["city"] + " " + row["state"] 


# create a new column in the dataframe with the result of the apply
ufos_df["location_name"] =  ufos_df.apply( create_location_str, axis=1)
ufos_df.head()

,datestr,city,state,shape,duration_description,location_name
0,3/23/17 07:50,Asheville,NC,cigar,2 minutes,Asheville NC
1,3/22/17 22:25,Frederick,MD,formation,4-5 seconds,Frederick MD
2,3/22/17 22:15,Dover,NH,triangle,5 minutes,Dover NH
3,3/22/17 21:30,North Richland Hills,TX,light,30 seconds,North Richland Hills TX
4,3/22/17 20:00,Sheboygan,WI,fireball,5-10 minutes,Sheboygan WI


In [62]:
## what happens when doing axis= 0
## column wise aggregation
def what_is_this(x):
    #print x
    #print x.__class__
    print max(x)
    
    
ufos_df.head().apply(what_is_this, axis=0)
ufos_df.head()["datestr"].max()

3/23/17 07:50
Sheboygan
WI
triangle
5-10 minutes


u'3/23/17 07:50'

In [64]:
## we want to get the longitude and latitude from the location string
# install it via
# sudo pip install geocoder
import geocoder

In [70]:
#dir(geocoder)
geocoder.google?
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [91]:
# Alternative way: get the unique locations first and then create the lon lat column
unique_locations=  ufos_df["location_name"].unique()
d = {}
for location in unique_locations:
    res = get_lon_lat_from_google(location)
    d[location] = res
    
ufos_df["lon_lat"] = ufos_df["location_name"].apply(lambda x: d[x])

3207


In [88]:
import time
cache = {}

def get_lon_lat_from_google(location_name):
    
    if location_name is in cache:
        return cache[location_name]
    try:
        res = geocoder.google(location_name)
        time.sleep(0.1)
        cache[location_name] = res
        #print res
        return (res.lng, res.lat)
    except:
        return None
    
# Several problems with using online APIs
# 1. sometimes your data is invalid, or their data is missing -> so use try except
# 2. it is very likely that we send a query to google for the same location multiple times
# API providers generally say: Please cache the results
# 3. APIs have limits (say 5k calls a day) and in addition, X number of requests per second
# if you use too many requests per second you might be blocked



In [87]:
lonlat = ufos_df["location_name"].apply(get_lon_lat_from_google)

KeyboardInterrupt: 

In [84]:
lonlat

0    (-82.5514869, 35.5950581)
1    (-77.4105409, 39.4142688)
2    (-70.8736698, 43.1978624)
3    (-97.2289029, 32.8342952)
4      (-87.71453, 43.7508284)
Name: location_name, dtype: object